In [3]:
import re
import numpy as np
import pandas as pd

### extract pancreas-relevant sentences in the findings section of the reports

In [595]:
rep = pd.read_csv('reports.csv')

In [602]:
in_anatomy = ['pancrea', 'uncus', 'uncinate']

In [603]:
ex_anatomy = ['airway', 'chest', 'trachea', 'bronch', 'carina', 'thorax', 'thoracic', 
              'lung', 'pulmonary', 'heart', 'cardia', 'pleura', 'mediastin', 'thyroid', 'breast', 
              'liver', 'hepatic', 'biliary', 'bile', 'gall', 'chole', 'spleen', 'splen', 
              'kidney', 'renal', 'nephro', 'adrenal', 'esophageal', 'stomach', 'gastric', 
              'gastro', 'intestine', 'colon', 'rectum', 'rectal', 'bowel', 'appendix', 
              'appendiceal', 'duodenal', 'iliac', 'ileum', 
              'jejunal', 'mesenteric', 'small intestine', 'small bowel', 'gi tract', 
              'ovary', 'adnexa', 'adnexal', 'uterus', 'uterine', 'vagina', 'pelvi', 'ureter', 'perineum', 
              'peritoneal', 'prostate', 'bladder', 
              'brain', 'bone', 'bony', 'osseous', 'spine', 'spinal', 'vertebr', 'thoraco', 
              'lumbar', 'skeletal', 'musculo', 'muscle', 'abdominal wall', 
              'gynecomastia', 'soft tissue', 'axilla', ' rib ', 'portal', 'porto', 'celiac', 
              'vascular', 'aorta', 'aortic', 'coronary', 'athero', 'common iliac', 'vena cava', 
              'vasculature', 'lymph']

In [597]:
results = []
for x in range(len(rep)):
    string = rep.iloc[x, 6].lower()
    if 'findings:' in string:
        m = re.search(r'findings:', string)
        string = string[m.start():]
        m = re.search(r'impression:', string)
        if m:
            string = string[:m.start()]
            if 'pancreas:' in string:
                m = re.search(r'pancreas:', string)
                string = string[m.start():]
                results.append(re.sub(r'\s+', ' ', string).strip())
            elif 'abdomen/pelvis:' in string:
                m = re.search(r'abdomen/pelvis:', string)
                string = string[m.start():]
                results.append(re.sub(r'\s+', ' ', string).strip())
            elif 'abdomen and pelvis:' in string:
                m = re.search(r'abdomen and pelvis:', string)
                string = string[m.start():]
                results.append(re.sub(r'\s+', ' ', string).strip())
            else:
                results.append(re.sub(r'\s+', ' ', string).strip())
        else:
            m = re.search(r'conclusion:|impression|impressions:', string)
            if m:
                string = string[:m.start()]
                if 'pancreas:' in string:
                    m = re.search(r'pancreas:', string)
                    string = string[m.start():]
                    results.append(re.sub(r'\s+', ' ', string).strip())
                elif 'abdomen/pelvis:' in string:
                    m = re.search(r'abdomen/pelvis:', string)
                    string = string[m.start():]
                    results.append(re.sub(r'\s+', ' ', string).strip())
                elif 'abdomen and pelvis:' in string:
                    m = re.search(r'abdomen and pelvis:', string)
                    string = string[m.start():]
                    results.append(re.sub(r'\s+', ' ', string).strip())
                else:
                    results.append(re.sub(r'\s+', ' ', string).strip())
            else:
                results.append('')
    else:
        results.append('')

In [ ]:
results_panc = []
for idx in range(len(results)):
    sentences = results[idx].replace('cm.', 'cm').replace('mm.', 'mm').split('. ')
    start = []
    stop = -1
    for i, s in enumerate(sentences):
        if re.search(r'(?<!peri)pancrea', s) or 'uncus' in s or 'uncinate' in s:
            start.append(i)
    if start:
        for i, s in enumerate(sentences):
            if i > start[-1]:
                for j in ex_anatomy:
                    if j in s:
                        stop = i
                        break
        if stop != -1:
            results_panc.append('. '.join(sentences[start[0]:stop])+'.')
            print('. '.join(sentences[start[0]:stop])+'.')
        else:
            results_panc.append('. '.join(sentences[start[0]:start[-1]+1])+'.')
            print('. '.join(sentences[start[0]:start[-1]+1])+'.')
    else:
        results_panc.append('no findings were found.')

In [616]:
rep['findings'] = results_panc

In [618]:
rep.to_csv('reports_pancreas_findings.csv', index=False)

### extract a single largest diameter per report

In [1040]:
df = pd.read_csv('output.csv')

In [ ]:
maxi = []
for i in range(len(df)):
    phrase = set([p.replace(' ', '') for p in df.iloc[i, 4:].tolist() if isinstance(p, str) and "[CLS]" not in p and "[PAD]" not in p])
    reg = r'\d.*?(?:cm|mm|(?<!sub)centimeter|millimeter)''
    tmp = []
    for p in phrase:
        if (len([p]) > 0) and (len([p]) < 40):
            for rx in re.findall(reg, p):
                tmp.append(rx)
    tmp = list(set(tmp))
    if tmp:
        ans = []
        for x in tmp:
            reg2 = r'(?:([0-9.]+)(?:(?:\(.*?\))?)(?:x|by|to|,|,and|and))?(?:([0-9.]+)(?:(?:\(.*?\))?)(?:x|by|to|,|,and|and))?([0-9.]+)(?:(?:\(.*?\))?)(?:-?)(?:(?:\(.*?\))?)(cm|mm|centimeter|millimeter)'
            rx2 = re.findall(reg2, x)
            if rx2:
                for z in range(len(rx2[0])-1):
                    y = rx2[0][z]
                    if y:
                        if (rx2[0][-1]=='cm') or (rx2[0][-1]=='centimeter'):
                            if not y.startswith('.'):
                                ans.append(float(y)*10)
                            elif y.startswith('.'):
                                ans.append(float(y[1:])*10)
                        elif (rx2[0][-1]=='mm') or (rx2[0][-1]=='millimeter'):
                            ans.append(float(y))
        if ans:
            maxi.append(max(ans))
        else:
            maxi.append(np.nan)
    else:
        maxi.append(np.nan)

In [1049]:
df['max'] = maxi

In [1051]:
df.to_csv('output_largest_diameter.csv', index=False)